In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import ddsp
from thesis import newt
import einops

from ddsp.colab.colab_utils import specplot
import gin

gin.enter_interactive_mode()

In [ ]:
n_samples = 16000
harm = newt.NEWTHarmonic(n_harmonics=1, n_outputs=3, sample_rate=16000, n_samples=n_samples)

# f0 = tf.zeros((2, 16000)) + 440
f0 = tf.linspace(440, 880, n_samples).reshape((1, -1, 1))
f0 = einops.repeat(f0, "b t 1 -> (b2 b) t 1", b2=2)
exciter = harm(f0)
exciter.shape

In [ ]:
import matplotlib.pyplot as plt

plt.plot(exciter[0, :100])

In [ ]:
import IPython.display


def play_audio(audio):
    IPython.display.display(IPython.display.Audio(audio, rate=16000))


play_audio(exciter[0, :, 0])

## Control module

In [ ]:
f0 = tf.zeros((2, 16000)) + 440
other_controls = tf.zeros((2, 16000)) + 1

In [ ]:
cm = newt.NEWTControlModule(hidden_size=30, embedding_size=10)
cm(f0, other_controls)

In [ ]:
layer = tf.keras.layers.Activation('identity')
output = layer([-3.0, -1.0, 0.0, 2.0])
# list(output.numpy())
output.numpy()

## NEWT waveshaper

In [ ]:
control_embedding_size = 8
batch_size = 2
control = tf.random.normal((batch_size, 100, control_embedding_size))

In [ ]:

shaper = newt.NEWTWaveshaper(
    n_waveshapers=3,
    control_embedding_size=control_embedding_size,
    shaping_fn_hidden_size=16,
)
y = shaper(exciter, control)
y.shape

In [ ]:
plt.plot(y[0])
play_audio(y[0,:])